In [25]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import random
from urllib.request import urlopen
import nltk
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [26]:
df = pd.read_csv('input_plot.csv')
th = pd.read_csv('thriller.csv')
g = th['Director'][1142:].values.tolist()
l = th['Plot_reduction'][1142:].values.tolist()

In [27]:
# df.head()
# df = df[df['Genre']!='unknown']
# df = df[df['Release Year']>1950]
# df[["Title","Release Year","Genre",'Wiki Page']].head()
# # df.columns
# # df.shape
# df1 = df
# df.iloc[0]['Wiki Page']
def stem(ge,q):
    m = 12
    index = g.index(ge)
    if(index in [1,3,7]):
        m = 10
    w = l[index].split(',')
    s = q.split()
    n = int(len(s)/m)
    
    for j in range(1,n):
        s = s[:j*m] + [w[random.randint(0,len(w)-1)]] + s[j*m:]
    return ' '.join(s)

In [56]:
movie_name = input('Enter a movie name : ')
movie_name = movie_name.title()
movie = ""
for i in range(len(movie_name)):
    if movie_name[i] == ' ':
        movie += '+'
    else:
        movie += movie_name[i]
print(movie)
url = 'https://www.google.com/search?q='+movie
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
response = requests.get(url,headers = headers)
soup = BeautifulSoup(response.text, 'lxml')
gen = soup.find('div',{'data-attrid':'subtitle'}).find('span').text.lower()
gen = gen.split()[2]
print('genre : ',gen)
search_div = soup.find_all(class_='rc')
for result in search_div: 
    url = result.a.get('href') 
    if('wiki' in url):
        break
print(url)
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')
url = "https://en.wikipedia.org"+soup.find('a',{'title':'Edit section: Plot'})['href']
print(url)
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')
plot = soup.find('textarea').text
print(plot)

Enter a movie name : enthiran
Enthiran
genre :  action/sci-fi
https://en.wikipedia.org/wiki/Enthiran
https://en.wikipedia.org/w/index.php?title=Enthiran&action=edit&section=1
== Plot ==
After a decade of research, the scientist K. Vaseegaran creates a sophisticated [[humanoid robot]] with the help of his assistants, Siva and Ravi, to commission it into the [[Indian Army]]. He introduces the robot, named [[Chitti (character)|Chitti]], at a robotics conference in [[Chennai]]. Chitti helps Sana, Vaseegaran's [[medical student]] girlfriend, cheat in her examination, then saves her from being assaulted by a group of thugs. Vaseegaran's mentor, Professor Bohra, is secretly engaged in a project to create similar [[Android (robot)|android]] robots for a German terrorist organisation, but has so far been unsuccessful. The terrorists give him a month to have the robots ready, otherwise they will kill him.

Vaseegaran prepares Chitti for an evaluation by the Artificial Intelligence Research and D

In [57]:
def func(plot,title):
    try:
        i = plot.index('==')
        plot = plot[i+len('=='):]
        i = plot.index('==')
        plot = plot[i+len('=='):]
    except:
        plot = plot
    while(1):
        i = plot.find('[[')
        if(i==-1):
            break
        j = plot.find(']]')
        plot = plot[:i] + plot[j+2:]
    while(1):
        i = plot.find('{{')
        if(i==-1):
            break
        j = plot.find('}}')
        plot = plot[:i] + plot[j+2:]
    while(1):
        i = plot.find('(')
        if(i==-1):
            break
        j = plot.find(')')
        plot = plot[:i] + plot[j+2:]
    p = ''

    for i in plot:
        if(i!='\n'):
            p += i
    plot = p
    stop_words = set(stopwords.words('english'))
    tagged_sentence = nltk.tag.pos_tag(plot.split())
    edited_sentence = [word for word,tag in tagged_sentence if tag != 'NNP' and tag != 'NNPS']
    example_sentence = ' '.join(edited_sentence)
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~0123456789'''
    no_punct = ""
    for char in example_sentence:
       if char not in punctuations:
           no_punct = no_punct + char
       else:
            no_punct += ' '
    example_sentence = no_punct
    tagged_sentence = nltk.tag.pos_tag(example_sentence.split())
    edited_sentence = [word for word,tag in tagged_sentence if tag not in ['NNP','NNPS','NNS','NN']]
    example_sentence = ' '.join(edited_sentence)
    example_sentence = example_sentence.lower()
    stop_words = set(stopwords.words('english')) 

    word_tokens = word_tokenize(example_sentence) 

    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filt = [ps.stem(filtered_sentence[i]) for i in range(len(filtered_sentence))]
    filt = ' '.join(filt)
    for i in g:
        if i in gen:
            filt = stem(i,filt)
    df1 = pd.DataFrame([title,gen,filt],['title','genre','Plot_reduction'],columns = [df.shape[0]]).T
    return df1

a = func(plot,movie_name)
movie_name

'Enthiran'

In [58]:
a['Plot_reduction'].values

array(['creat sophist introduc name help girlfriend save assault secretli engag creat similar german terrorist far unsuccess give readi otherwis kill head stab convinc use militari prove wrong deploy rescu burn save includ name bath asham seen nake flee hit kill ask one modifi enabl understand human agre near becom angri demonstr manifest use success help sister give congratul allow pass tell start romant congratul kiss realis explain chitti sadden yet still conduct indian follow frustrat chop dump landfil visit retriev reassembl albeit damag emb red insid reconstruct convert aggress unempathet gatecrash wed creat use attempt kill caus inform acquir human reproduc marri human give preprogram refus eventu find enter stop nearli appear ensu robot lead mani eventu captur use magnet access intern instruct self remov red calm sentenc caus robot explain caus deviant show instal red releas order dismantl ask dismantl say goodby forgiv dismantl set curiou ask dismantl respond'],
      dtype=ob

In [59]:
pd.concat([df,a])

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Plot_reduction,title,genre
0,1971.0,The Horsemen,American,John Frankenheimer,"Omar Sharif, Leigh Taylor-Young, Jack Palance",adventure,https://en.wikipedia.org/wiki/The_Horsemen_(19...,"Uraz (Omar Sharif), the son of Tursen (Jack Pa...",stabl retir journey feudal risk live seek prim...,NaN,NaN
1,1971.0,The Light at the Edge of the World,American,Kevin Billington,"Kirk Douglas, Yul Brynner, Samantha Eggar",adventure,https://en.wikipedia.org/wiki/The_Light_at_the...,The year is 1865. Will Denton (Kirk Douglas) i...,jade american escap troubl two mend broken fai...,NaN,NaN
2,1971.0,Sometimes a Great Notion,American,Paul Newman,"Paul Newman, Henry Fonda, Lee Remick, Michael ...",adventure,https://en.wikipedia.org/wiki/Sometimes_a_Grea...,"The economic stability of Waconda, Oregon, is ...",econom threaten local call larg independ urg j...,NaN,NaN
3,1972.0,Napoleon and Samantha,American,Bernard McEveety,"Michael Douglas, Jodie Foster",adventure,https://en.wikipedia.org/wiki/Napoleon_and_Sam...,Eleven-year-old Napoleon lives with his grandf...,eleven old live adopt name meet old take back ...,NaN,NaN
4,1972.0,The Poseidon Adventure,American,Ronald Neame,"Gene Hackman, Ernest Borgnine, Shelley Winters...","disaster film, adventure",https://en.wikipedia.org/wiki/The_Poseidon_Adv...,"The S.S. Poseidon, an ocean liner slated for r...",ocean slate despit fear new repres go full sav...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
9430,2011.0,White as Snow,Turkish,Selim Güneş,"Hakan Korkmaz, Sinem İslamoğlu & Gürkan Piri O...",drama,https://en.wikipedia.org/wiki/White_as_Snow_(f...,Hasan is a twelve-year-old boy living with his...,twelv old two younger struggl surviv sinc fath...,NaN,NaN
9431,2011.0,Once Upon a Time in Anatolia,Turkish,Nuri Bilge Ceylan,"Yılmaz Erdoğan, Taner Birsel & Ufuk Karaali",drama,https://en.wikipedia.org/wiki/Once_Upon_a_Time...,"Through the night, three cars carry a small gr...",three carri small gendarmeri two homicid suspe...,NaN,NaN
9432,2013.0,Selam,Turkish,Levent Demirkale,"Bucin Abdullah, Selma Alispahic, Tina Cvitanov...",drama,https://en.wikipedia.org/wiki/Selam_(film),The film opens with a Senegalese boy named Kha...,open senegales name told littl fallen two deba...,NaN,NaN
9433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,smuggl pre appreci die tri foil take orchestr ...,Ayan,action


In [60]:
a

,title,genre,Plot_reduction
9434,Enthiran,action/sci-fi,creat sophist introduc name help girlfriend sa...


In [9]:
gen = ['sci_fi','fantasy','crime','adventure','music']
cons = pd.DataFrame()
def func2(genre,gen_name):
    global cons
    genre = genre[genre['Plot_reduction']!= np.nan]
    th_plot = genre['Plot_reduction']
    th_plot = th_plot.dropna()

    th_plot_80,th_plot_20 = train_test_split(th_plot,train_size = int(len(th_plot)*0.8),test_size = len(th_plot)-int(len(th_plot)*0.8))

    if(gen_name in gen):
        vectorizer = TfidfVectorizer(min_df = 50,stop_words='english',max_df = 0.9,max_features = 500)
    else:
        vectorizer = TfidfVectorizer(min_df = 200,stop_words='english',max_df = 0.9,max_features = 500)
    th_plot = th_plot_80.append(pd.Series(th_plot_20))

    bagofwords = vectorizer.fit_transform(th_plot)
    feature_names = vectorizer.get_feature_names()
    dense = bagofwords.todense()
    denselist = dense.tolist()
    df = pd.DataFrame(denselist, columns=feature_names)
    rat = df.iloc[df.shape[0]-len(th_plot_20):]

    ncomp = 15
    svd = TruncatedSVD(n_components = ncomp)
    lsa = svd.fit_transform(bagofwords)
    lsa
    diction30 = vectorizer.get_feature_names()

    topic = ['Topic '+str(i) for i in range(1,ncomp+1)]
    
    encoding_matrix = pd.DataFrame(svd.components_,index = topic).T
    encoding_matrix['words'] = diction30
    diction30 = encoding_matrix.sort_values(by=['Topic 1'],ascending = False)
    diction30.index = diction30['words']
    diction30 = diction30.drop(['words'],axis = 1)
    
    rat = rat.T
    rat.index.set_names(['words'],inplace = True)
    comp = pd.concat([rat,diction30], axis=1, join='inner').T
    c = cosine_similarity(comp,comp)
    final = pd.DataFrame(c[len(c[0])-ncomp:,:comp.shape[0]-ncomp],index = topic)
    
    
    final['sum'] = final.sum(axis = 1,skipna = True)
    cons = final['sum']
    return final.idxmax(axis = 0,skipna = True)['sum']

In [10]:
gen = ['sci_fi','fantasy','crime','adventure','music']
ncomp =15
def func1(genre,summa,gen_name,title):
    global cons
    genre = genre[genre['Plot_reduction']!= np.nan]
    th_plot = genre['Plot_reduction']
#     print(th_plot.head(1))
    th_plot = th_plot.append(pd.Series(summa,index = range(len(th_plot),len(th_plot)+len(summa))))
#     print(len(summa),th_plot.shape[0])
    th_plot = th_plot.dropna()
    if(gen_name in gen):
        vectorizer = TfidfVectorizer(min_df = 50,stop_words='english',max_df = 0.9,max_features = 500)
    else:
        vectorizer = TfidfVectorizer(min_df = 200,stop_words='english',max_df = 0.9,max_features = 500)
    bagofwords = vectorizer.fit_transform(th_plot)
    feature_names = vectorizer.get_feature_names()
    dense = bagofwords.todense()
    denselist = dense.tolist()
    df2 = pd.DataFrame(denselist, columns=feature_names)
    rat = df2.iloc[df2.shape[0]-len(summa):]
    rat = rat.T
    rat.index.set_names(['words'],inplace = True)

    svd = TruncatedSVD(n_components = ncomp)
    lsa = svd.fit_transform(bagofwords)
    
    diction30 = vectorizer.get_feature_names()
    
    top = func2(genre,gen_name)
    topic = ['Topic '+str(i) for i in range(1,ncomp+1)]
    
    encoding_matrix = pd.DataFrame(svd.components_,index = topic).T
    encoding_matrix['words'] = diction30
    diction30 = encoding_matrix.sort_values(by=['Topic 1'],ascending = False)
    diction30.index = diction30['words']
    topic.remove(top)
    diction30 = diction30.drop(topic,axis = 1)
    diction30 = diction30.drop(['words'],axis = 1)
    
    comp = pd.concat([rat,diction30], axis=1, join='inner').T
    comp = comp.drop(['tell','ask','come','want','make','say','young'], axis=1, errors='ignore')
    if gen_name not in ['horror','thriller','action']:
        comp = comp.drop(['kill'],axis = 1,errors = 'ignore')

    c = cosine_similarity(comp,comp)
    temp = pd.DataFrame(c[len(c[0])-1],index = title)
    temp.rename(columns = {0:gen_name},inplace = True)
    return temp.T
#     print(c)

In [13]:
df = pd.read_csv("movie_info1.csv")

In [14]:
action = df[df.apply(lambda x: "action" in x['Genre'].lower(),axis = 1)]
drama = df[df.apply(lambda x: "drama" in x['Genre'].lower() , axis=1)]
fantasy = df[df.apply(lambda x: "fantasy" in x['Genre'].lower(),axis = 1)]
horror = df[df.apply(lambda x: "horror" in x['Genre'].lower(),axis = 1)]
sci_fi = df[df.apply(lambda x: "sci" in x['Genre'].lower(),axis = 1)]
romance = df[df.apply(lambda x: "romance" in x['Genre'].lower() or "romantic"  in x['Genre'].lower(),axis = 1)]
thriller = df[df.apply(lambda x: "thriller" in x['Genre'] or "mystery" in x['Genre'].lower(),axis = 1)]
crime = df[df.apply(lambda x: "crime" in x['Genre'].lower(),axis = 1)]
adventure = df[df.apply(lambda x: "adventure" in x['Genre'].lower(),axis = 1)]

In [15]:
genre_df = [action,adventure,crime,drama,fantasy,horror,romance,sci_fi,thriller]
genre_nm = ['action','adventure','crime','drama','fantasy','horror','romance','sci_fi','thriller']
def func5(plot,title):
    df = pd.DataFrame()
    df1 = pd.DataFrame()
    for j in range(len(plot)):
        for i in range(len(genre_df)):
            a = func1(genre_df[i],[plot[j]],genre_nm[i],[title[j],'temp'])
            df = pd.concat([df,a])
        df = df.drop(['temp'],axis = 1,errors = 'ignore')
        df1 = pd.concat([df1,df],axis = 1)
        df = pd.DataFrame()
    
    return df1    

In [17]:
#ans = func5(a['Plot_reduction'],a['title'])

In [61]:
pd.concat([df,a]).to_csv('input_plot.csv')
pd.concat([df,a]).to_csv('input_plot.csv',index=False)